In [4]:
import csv
import json
from typing import TypedDict
from collections import defaultdict

In [2]:
file = "../../data/toflit18_all_flows.csv"
product_classification = "product_sitc_simplEN"

port_francs = ["Lorient", "Bayonne", "Marseille", "Dunkerque"]

data_by_port_francs = {p:{} for p in port_francs}

In [5]:
class DataPoint(TypedDict):
    product: str 
    import_dk: int
    import_dkb: int
    export_dk: int
    export_dkb: int

In [8]:
report_port_francs = {p:defaultdict(dict) for p in port_francs}
report_others = {p:defaultdict(dict) for p in port_francs}
import_by_product_by_partner = {p:defaultdict(dict) for p in port_francs}
export_by_product_by_partner  = {p:defaultdict(dict) for p in port_francs}
def import_partner_class(partner):
    if partner in ["Asie", "Afrique", "Amériques"]:
        return "colonies"
    if partner == "France":
        return "France"
    return 'Monde'


with open(file, "r") as muerte:
    reader = csv.DictReader(muerte)
    for i, row in enumerate(reader):
        if row["year"] == "1789":
            product = row[product_classification]
            region = row["customs_region_grouping"]
            partner = row["partner_simplification"]
            value = float(row["value"]) if row["value"] else 0.0
            office = row["customs_office"] if row["customs_office"] != "Port franc De Bayonne" else "Bayonne"

            # process mirrors flows : France imports from Port France 
            if row["export_import"] == "Imports" and partner in port_francs:
                if product:
                    if region not in report_others[partner][product]:
                        report_others[partner][product][region] = {}
                    report_others[partner][product][region]["product"] = product
                    report_others[partner][product][region]["region"] = region
                    report_others[partner][product][region]["import"] = report_others[partner][product][region].get("import", 0) + value

            # process mirrors flows : France exports to Dunkerque
            if row["export_import"] == "Exports" and partner in port_francs:
                if product:
                    import_by_product_by_partner[partner][product]["France"] = import_by_product_by_partner[partner][product].get("France", 0 ) + value

            # flows declared by ports francs
            if row["customs_office"] in port_francs:
                if product:
                    report_port_francs[office][product]["product"] = product
                    partner = import_partner_class(row["partner_grouping"])
                    if row["export_import"] == "Exports":
                        report_port_francs[office][product]["export_dk"] = report_port_francs[office][product].get("export_dk", 0) + value
                        export_by_product_by_partner[office][product][partner] = export_by_product_by_partner[office][product].get(partner,0) + value
                    elif row["export_import"] == "Imports":
                        report_port_francs[office][product]["import_dk"] = report_port_francs[office][product].get("import_dk", 0) + value       
                        import_by_product_by_partner[office][product][partner] = import_by_product_by_partner[office][product].get(partner,0) + value
                   

In [ ]:
import_by_product_by_partner
# partners

In [13]:
all_data = {p:[] for p in port_francs}
all_exp_to_fr = 0
get_products = lambda d: set([p for (port,product_dict) in d.items() for p in product_dict.keys()])
products = get_products(report_port_francs) | get_products(report_others)

for product in products:
    for port in port_francs:
        flows = report_port_francs[port][product] if product in report_port_francs[port] else {}
        is_colonial = False
        # IMPORTS
        total_declared_import = flows.get("import_dk",0) + import_by_product_by_partner[port][product].get("France",0)
        if product in import_by_product_by_partner[port]:
            for (partner, value) in import_by_product_by_partner[port][product].items():
                if partner == "colonies":
                    is_colonial = value/total_declared_import >= 0.5
                all_data[port].append({"value":value, "product": product, "partner" : partner, "group": "import"})

        if product in export_by_product_by_partner[port]:
            for (partner, value) in export_by_product_by_partner[port][product].items():
                all_data[port].append({"value":value, "product": product, "partner" : partner, "group": "import"})
        
        export_to_fr = sum([dp['import'] for (region,dp) in report_others[port].get(product,{}).items()])
        all_data[port].append({"value":export_to_fr, "product": product, "partner" : "France", "group": "export"})
        
        total_declared_export = flows.get("export_dk", 0)
        total_declared_import = flows.get("import_dk",0) + import_by_product_by_partner[port][product].get("France",0)
        unknown_export = total_declared_import - total_declared_export

        if port != "Dunkerque":
            all_data[port].append({"value":unknown_export , "product": product, "partner" : "Fraude ?", "group": "export"})
        else:
            if is_colonial:
                all_data[port].append({"value":unknown_export , "product": product, "partner" : "Fraude ?", "group": "export"})
            else:
                all_data[port].append({"value":unknown_export , "product": product, "partner" : "Re-exp ou fraude ?", "group": "export"})


In [14]:
all_data

{'Lorient': [{'value': 30861.0,
   'product': 'Chemical products',
   'partner': 'Monde',
   'group': 'import'},
  {'value': 1177.5,
   'product': 'Chemical products',
   'partner': 'France',
   'group': 'import'},
  {'value': 67324.5,
   'product': 'Chemical products',
   'partner': 'France',
   'group': 'export'},
  {'value': 114605.7989,
   'product': 'Chemical products',
   'partner': 'Fraude ?',
   'group': 'export'},
  {'value': 180017.035,
   'product': 'Crude materials, inedible, except combustibles',
   'partner': 'Monde',
   'group': 'import'},
  {'value': 119.999998,
   'product': 'Crude materials, inedible, except combustibles',
   'partner': 'France',
   'group': 'import'},
  {'value': 303793.30011473,
   'product': 'Crude materials, inedible, except combustibles',
   'partner': 'France',
   'group': 'export'},
  {'value': 256043.26507810003,
   'product': 'Crude materials, inedible, except combustibles',
   'partner': 'Fraude ?',
   'group': 'export'},
  {'value': 0,
   '

In [ ]:
with open("data/report_dunkerque.json", "w") as f:
    json.dump(report_dk, f)

In [ ]:
with open("data/report_other_regions.json", "w") as f:
    json.dump(report_others, f)

In [75]:
with open("data/import_export_ports_francs.json", "w") as f:
    json.dump(all_data, f)